## Testing for fun

In [4]:
import numpy as np
import pandas as pd
from datasets import Dataset, DatasetDict
from transformers import Trainer
from transformers import AutoConfig
from torch import nn
from transformers.modeling_outputs import SequenceClassifierOutput
from transformers.models.bert.modeling_bert import BertModel, BertPreTrainedModel

# https://github.com/huggingface/transformers/blob/65659a29cf5a079842e61a63d57fa24474288998/src/transformers/models/bert/modeling_bert.py#L1486

from transformers import AutoTokenizer

transformer_name = "league_kaggle_model" # put name of directory with the model here
tokenizer = AutoTokenizer.from_pretrained(transformer_name, use_fast=True)

def tokenize(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length')


class BertForSequenceClassification(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.bert = BertModel(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)
        self.init_weights()
        
    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None, labels=None, **kwargs):
        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            **kwargs,
        )
        cls_outputs = outputs.last_hidden_state[:, 0, :]
        cls_outputs = self.dropout(cls_outputs)
        logits = self.classifier(cls_outputs)
        loss = None
        if labels is not None:
            loss_fn = nn.CrossEntropyLoss()
            loss = loss_fn(logits, labels)
        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

config = AutoConfig.from_pretrained(
    transformer_name,
    #num_labels=len(labels),
    num_labels=2
)

model = (
    BertForSequenceClassification
    .from_pretrained(transformer_name, config=config)
)

trainer = Trainer(
    model=model,
)

In [5]:
cols = ["text", "label"]
data = [["soup",0],
        ["this is a soup", 0],
        ["in the library", 0],
        ["Motherfucking noob",1],
        ["AHAHHHAHHASDHADHU",0],
        ["bread and butter",0],
        ["soup is bad",0],
        ["I hope you beat your children",1],
        ["die soup",0], 
        ["I hope you die",1],
        ["I hope your kids die",1],
        ["I hope your kids", 0],
        ["I hope your kids soup", 0],
        ["die udyr", 1],
        ["die kids", 1],
        ["I hope your lose",0],
        ["I hope your succeed",0],
       ]
test = pd.DataFrame(data, columns = cols)

# Convert the DataFrame to a Dataset
test_dataset = Dataset.from_pandas(test)
#test_dataset.reset_index(inplace=True, drop=True)

dota_ds=DatasetDict()
dota_ds['test'] = Dataset.from_pandas(test)
#dota_ds.reset_index(inplace=True, drop=True)


# Now, map the tokenization function to the dataset
# Since your key for the dataset is 'test', make sure you access 'test' from dota_ds
test_ds_dota = dota_ds['test'].map(
    tokenize,
    batched=True,
    # remove_columns=['message'],  # Removing 'message' column after tokenization
)

# Convert to pandas DataFrame (for inspection)
test_ds_dota_df = test_ds_dota.to_pandas()

# Output the tokenized dataset
# print(test_ds_dota_df)


# Make predictions
output_dota = trainer.predict(test_ds_dota)

# Print or inspect the output
# print(output_dota)

output_dota.predictions

chats = test_dataset["text"]
labels = np.argmax(output_dota.predictions, axis=-1)
pd.DataFrame(labels, chats)

Map:   0%|          | 0/17 [00:00<?, ? examples/s]

,0
soup,0
this is a soup,1
in the library,1
Motherfucking noob,1
AHAHHHAHHASDHADHU,0
bread and butter,1
soup is bad,1
I hope you beat your children,1
die soup,0
I hope you die,0
